In [ ]:
from pyspark import SparkContext, SparkConf
import os, shutil

In [ ]:
#os.environ["PYSPARK_PYTHON"] = "/usr/bin/env python3"
master = "local[4]"
appName = "task1"
conf = SparkConf().setAppName(appName).setMaster(master)

In [ ]:
sc = SparkContext(conf=conf)

In [ ]:
sc.setLogLevel("WARN")

## Creating RDD

In [ ]:
rdd = sc.textFile("data/geotweets.tsv")

### Creating a sample RDD for testing

In [ ]:
sampled_rdd = rdd.sample(False, 0.1, 5)

### Creating array splitting on tabs

In [ ]:
rdd_list = rdd.map(lambda x: x.split('\t'))

In [ ]:
sampled_rdd_list = sampled_rdd.map(lambda x: x.split('\t'))

# Task 1

In [ ]:
number_of_tweets = rdd.count()
print(number_of_tweets)

In [ ]:
number_of_users = rdd_list.map(lambda x: x[6]).distinct().count()
print(number_of_users)

In [ ]:
number_of_countries = rdd_list.map(lambda x: x[1]).distinct().count()
print(number_of_countries)

In [ ]:
number_of_places = rdd_list.map(lambda x: x[4]).distinct().count()
print(number_of_places)

In [ ]:
#number_of_languages = rdd_list.map(lambda x: (x, 1) ).reduce(lambda a, b: a+b)
number_of_languages = rdd_list.map(lambda x: x[5]).distinct().count()
print(number_of_languages)

In [ ]:
minimum_latitude = rdd_list.map(lambda x: float(x[11])).reduce(lambda a, b: min(a,b))
print(minimum_latitude)

In [ ]:
minimum_longitude = rdd_list.map(lambda x: float(x[12])).reduce(lambda a, b: min(a,b))
print(minimum_longitude)

In [ ]:
maximum_latitude = rdd_list.map(lambda x: float(x[11])).reduce(lambda a, b: max(a,b))
print(maximum_latitude)

In [ ]:
maximum_longitude = rdd_list.map(lambda x: float(x[12])).reduce(lambda a, b: max(a,b))
#alsjdk
print(maximum_longitude)

In [ ]:
tweet_text = rdd_list.map(lambda x: x[10])
print(tweet_text)

In [ ]:
tweet_in_characters = tweet_text.map(lambda x: (len(x)))
average_tweet_in_characters = tweet_in_characters.mean()
print(average_tweet_in_characters)

In [ ]:
tweet_in_words = tweet_text.map(lambda x: len(x.split(' ')))
average_tweet_in_words = tweet_in_words.mean()
print(average_tweet_in_words)

### Combining results to RDD and writes to file

In [ ]:
results = sc.parallelize([number_of_tweets, number_of_users,\
                        number_of_countries, number_of_places,\
                        number_of_languages, minimum_latitude,\
                        minimum_longitude, maximum_latitude, maximum_longitude,\
                        average_tweet_in_characters, average_tweet_in_words])
results = results.coalesce(1)
resultsPath = 'results/result_1.tsv'
if os.path.isdir(resultsPath):
    shutil.rmtree(resultsPath)
results_tsv = results.saveAsTextFile(resultsPath)

# Task 2

In [ ]:
from operator import add
new_rdd = sampled_rdd_list.map(lambda x: (x[5], 1)).countByKey().items()
sorted_dict = sorted(new_rdd, key=lambda x: x[1])
sorted_dict.reverse()
new_rdd2 = sampled_rdd_list.map(lambda x: (x[5], 1)).reduceByKey(add)

In [ ]:
print(sorted_dict)
print("")
#lakjslkdj
print(new_rdd2.sortBy(lambda x: x[1]).collect())